In [34]:
# | echo: false
# | output: false

import sys
import os

sys.path.append(os.path.abspath("../../"))

%reload_ext autoreload
%autoreload 2

from python_utils.get_browser import get_browser_page_async

class UsernameChecker:
    def __init__(self, username):
        self.username = username
        self.results = {}
        self.platforms = [
            {
                "name": "LinkedIn Company",
                "uri": f"https://www.linkedin.com/company/{username}/",
                "not_available_match": ["Uh oh, we can", "seem to find the page you", "Try going back to the previous page"],
                "available_match": [username, "Skip to main content", "followers", "Follow"],
            },
            {
                "name": "LinkedIn User",
                "uri": f"https://www.linkedin.com/in/{username}/",
                "not_available_match": ["This profile may be private", "may not exist", "may be private"],
                "available_match": [username, "Skip to main content", "followers", "Follow"],
            }
        ]

    async def check_availability_async(self):
        for platform in self.platforms:
            try:
                page, close = await get_browser_page_async()
                await page.goto(platform['uri'], wait_until='networkidle')
                # Wait for 1 second after load
                await page.wait_for_timeout(1000)
                text_content = (await page.evaluate("document.body.innerText") or '').lower()
                not_available_match = any((i or '').lower() in text_content for i in platform["not_available_match"])
                available_match = any((i or '').lower() not in text_content for i in platform["available_match"])
                if not_available_match and available_match:
                    self.results[platform['name']] = {
                        'available': True,
                        'message': f"{self.username}: ✅ Available",
                        'url': platform['uri']
                    }
                else:
                    self.results[platform['name']] = {
                        'available': False,
                        'message': f"{self.username}: ❌ Taken",
                        'url': platform['uri']
                    }
                await close()
            except Exception as e:
                self.results[platform['name']] = {
                    'available': None,
                    'error': str(e)
                }
        return self.results

In [35]:
# Create checker instance
checker = UsernameChecker("toknowqwertyai")

# Check availability
await checker.check_availability_async()

{'LinkedIn Company': {'available': True,
  'message': 'toknowqwertyai: ✅ Available',
  'url': 'https://www.linkedin.com/company/toknowqwertyai/'},
 'LinkedIn User': {'available': False,
  'message': 'toknowqwertyai: ❌ Taken',
  'url': 'https://www.linkedin.com/in/toknowqwertyai/'}}